# Object Tracking Using SIFT

This notebook is tied to the previous notebook where I recreated the SIFT detector. Now I'll use the SIFT detector to see if I can track objects or patterns within an image. I'll be using the same traffic video that I did before

In [46]:
import cv2
import numpy as np
from IPython.display import display, Video
from matplotlib import pyplot as plt
from contextlib import contextmanager
from tqdm.notebook import tqdm

video_file = '../../data/traffic-video.mp4'
intermediate_file_template = '../../data/traffic-video-{step}.mp4'

Video(video_file)

Let's first see if we can track the grayscale version of the video file. So we'll convert the video to grayscale first. We need to grab the frame size and framerate from our previous capture. Since we'll be writing a lot of video, I'll create some functions to handle most of the common routines

In [58]:
@contextmanager
def video_pipe(from_file, to_file, fourcc='avc1', isColor=True):
    # Create capture and get params
    video_capture = cv2.VideoCapture(from_file)
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    framerate = video_capture.get(cv2.CAP_PROP_FPS)
    
    # Create writer with params
    fourcc = cv2.VideoWriter_fourcc(*fourcc)
    video_writer = cv2.VideoWriter(to_file, 
                                   fourcc, framerate, 
                                   (width, height), 
                                   isColor=isColor)
    
    # Yield capture and writer
    yield (video_capture, video_writer)
    
    # Close capture and writer
    video_writer.release()
    video_capture.release()
    
def frame_iter(capture, desc='Processing Frames'):
    frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    progress = tqdm(desc=desc, total=frames)
    has_frames, frame = capture.read()
    while has_frames:
        yield frame
        progress.update()
        has_frames, frame = capture.read()
    
grayscale_file = intermediate_file_template.format(step='grayscale')

# Convert frames to grayscale
with video_pipe(video_file, grayscale_file, isColor=False) as (cap, writer):
    for frame in frame_iter(cap):
        grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        writer.write(grayscale)

# Display new Video
Video(grayscale_file)

Processing Frames:   0%|          | 0/750 [00:00<?, ?it/s]

Next, we'll apply OpenCV's SIFT detector to each frame. In this example I'm simply going to run the detection and save the frame, so we can play it back and see all of the points.

In [67]:
number_of_features = 0

keypoints_file = intermediate_file_template.format(step='keypoints')

sift = cv2.SIFT.create(number_of_features)

with video_pipe(grayscale_file, keypoints_file) as (cap, wrt):
    for frame in frame_iter(cap):
        keypoints = sift.detect(frame)
        keypoints_frame = cv2.drawKeypoints(frame, keypoints, 0, 
            flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        wrt.write(keypoints_frame)
        
Video(keypoints_file)

Processing Frames:   0%|          | 0/750 [00:00<?, ?it/s]